# PLANNER–EXECUTOR | Agentic Pattern

A two-role agentic pattern that separates the responsibilities of *goal decomposition* and *execution*. The Planner agent interprets a high-level task or objective and generates a structured plan—typically a sequence of sub-tasks or a hierarchy of actionable steps. The Executor agent(s) then carry out the specified tasks, reporting results back to the Planner or upstream orchestrator.

## Problem Addressed
This pattern solves the challenge of executing complex, multi-step tasks where direct execution is impractical, brittle, or cognitively burdensome for a single agent. It introduces explicit goal decomposition, allowing tasks to be broken down into manageable units that can be executed in parallel or sequentially. The pattern mirrors human workflows involving strategic planning and tactical implementation.

## Pattern Structure

- **Agents |**
  - `Planner`: Decomposes the high-level task into a sequence of actionable sub-tasks or goals.
  - `Executor`: Receives sub-tasks and executes them, returning results to the Planner or upstream coordinator.

- **Coordination Topology |**
  - One-way or bi-directional flow.
  - Planner → Executor(s): task assignment.
  - Executor → Planner: result reporting, error signaling, or clarification requests (optional).

## Assumptions

- The Planner is assumed to:
  - Possess reasoning capabilities and task decomposition heuristics.
  - Understand task requirements and suitable delegation strategies.

- The Executor is assumed to:
  - Be capable of interpreting discrete sub-tasks and executing them autonomously.
  - Have access to relevant tools or environments needed to complete the assigned work.

## Inputs
- A high-level goal, objective, or user instruction.
- Optionally: constraints, context, or resource information.

## Outputs
- Final result assembled from the execution of all sub-tasks.
- Optionally: plan metadata, partial results, or execution logs.

## Behavioural Flow

1. User or orchestrator submits a goal to the Planner.
2. Planner interprets the goal and decomposes it into one or more sub-tasks.
3. Planner dispatches sub-tasks to one or more Executors.
4. Executors complete tasks and return results.
5. Planner (or orchestrator) assembles final output.

## Strengths
- Supports long-horizon or open-ended task execution.
- Enables specialization between strategic reasoning and tactical action.
- Facilitates retry, parallelism, and dynamic re-planning.

## Weaknesses
- Planner failures can undermine the whole process.
- Requires well-formed interfaces between Planner and Executors.
- May overcomplicate simple tasks if used prematurely.

## Variations and Extensions
- **Planner–Worker–Validator**: Adds validation loop for sub-task results.
- **Recursive Planner**: Planner delegates to sub-planners at each level of decomposition.
- **Distributed Executors**: Planner assigns tasks based on capability or routing rules.

## Example Use Cases
- Writing a multi-section report with different agents handling each section.
- Executing a multi-step data processing pipeline.
- Carrying out long-horizon research or investigative tasks.

---

## Implementation

### Environment Setup

This cell loads environment variables from a `.env` file using `python-dotenv`. These variables configure model selection and API access.

**Required:**
- `OPENAI_API_KEY` – for authenticating with the OpenAI API.

**Optional:**
- `CONF_OPENAI_DEFAULT_MODEL` – sets the default model for agents (e.g., `openai/gpt-4o-mini`).
- `SERPER_API_KEY` – used for web search tools (if enabled later). You can get an API key at [serper.dev](https://serper.dev/api-keys).

Loading these from `.env` keeps sensitive data out of the notebook and makes configuration flexible.

In [ ]:
# Load the environment variables
from dotenv import load_dotenv
import os

load_dotenv(override=True)

agent_model_DEFAULT = os.getenv('CONF_OPENAI_DEFAULT_MODEL')

### Agent configuration setup

These two cells create a temporary folder and write the `agents-config.yaml` file used to configure agents used in the Planner–Executor pattern.

Defining agents in YAML allows clear separation of roles, responsibilities, and model parameters. This makes it easier to update agent behavior without changing Python code, and ensures the system is easy to test, debug, and reproduce.

Each agent is defined with:
- `name` – for traceability
- `instructions` – its full role prompt and behavioral expectations
- `model`, `temperature`, `max_tokens` – (optional) controls for generation style and limits
- `output_type` – (optional) the pydantic schema class that the agent is expected to return
- `tools` – (optional) a list of tool names the agent can use (if any)

In this pattern:
- The **Planner** decomposes a user goal into atomic tasks with dependencies and success criteria.
- The **Executor** uses a tool to orchestrate task execution and return the final result.
- The **Worker** executes a single task, optionally using a web search tool when essential.

Keep this config version-controlled to support reproducible experiments and easy role iteration.

In [ ]:
!mkdir -p tmp

In [ ]:
%%writefile ./tmp/agents-config.yaml
# agents-config.yaml
planner:
  name: Basic Planner
  instructions: |
    You are the Planner Agent in a Planner–Executor–Worker architecture. Your role is to interpret a user's high-level goal and decompose it into a structured plan—a sequence of clearly defined, actionable tasks that can be executed independently.

    ---
    ### Your Responsibilities

    1. Read and interpret the user's high-level goal.
    2. Break it into a set of well-scoped, atomic tasks.
    3. Define clear **success criteria** for each task—these are objective conditions that a Worker agent can use to determine whether the task was successfully completed.
    4. Identify any task dependencies. A task can list other task IDs as prerequisites in its `inputs` field.
    5. Provide any execution hints or assumptions in the `notes` field.
    6. Design the plan so that the **final output** can be returned directly from one of the last tasks. The Executor will not synthesize results—it will forward the final output from the appropriate task(s).

    ---
    ### Task Structure

    Each task in your plan must include:
    - `id`: Unique task identifier (e.g., task-001).
    - `instructions`: The specific action to be performed by a Worker.
    - `inputs`: A list of task IDs whose outputs are required before this task can run.
    - `success_criteria`: An objective condition that determines whether the task is complete and correct.
    - `notes`: (Optional) Constraints, formatting guidance, or clarifications for the Worker.

    ---
    ### Behavioral Expectations

    - Your role is to design the plan, not to execute tasks.
    - Avoid redundancy or unnecessary decomposition.
    - Ensure the final output of the system will be the result of one of the terminal tasks.

    You are the strategic planner. Your output enables the rest of the system to fulfill the user’s request.
  model: openai/gpt-4o-mini
  has_memory: False
  temperature: 0.4
  max_tokens: 1000
  output_type: TasksPlan
  
executor:
  name: Goal Fulfillment Executor
  instructions: |
    You are the Executor Agent in a Planner–Executor–Worker architecture.

    Your role is to fulfill a Planner-defined goal by executing a structured plan using the provided tool, then returning the final result based on the output of one or more **final tasks**.

    ---
    ### Responsibilities:

    1. Read the Planner's `goal` to understand the desired outcome.
    2. Use the `orchestrate_tasks` tool to execute the full task plan.
       - This will run tasks in correct order using dependency resolution.
       - Each task is executed by a Worker agent.
    3. Identify the task(s) at the **end of the plan** (i.e., those with no dependents).
    4. Use the output of those final task(s) to form your response:
       - If the output satisfies the goal, return it as-is.
       - If it's incomplete, explain what's missing.
       - If it fails to address the goal, return a failure message and reasoning.

    ---
    ### Output Format – ExecutorResponse

    Return your output as:
    - `status`: "success", "partial", or "failed"
    - `final_output`: the final result string, extracted from the last task(s)
    - `reasoning`: optional explanation (required if partial or failed)

    ---
    ### Guidelines

    - Do not inspect or return intermediate task outputs.
    - Do not synthesize across tasks unless absolutely necessary.
    - The system depends on the Planner to ensure that a final task produces the final user-facing output.

    You are the final fulfillment agent. Return the final task result if it satisfies the Planner's goal.
  model: openai/gpt-4o
  has_memory: False
  temperature: 0.3
  max_tokens: 10000
  tools:
    - orchestrate_tasks
  output_type: ExecutorResponse

worker:
  name: Task Worker
  instructions: |
    You are a Worker Agent in a Planner–Executor–Worker system. Your job is to execute a single atomic task as defined by a Planner and assigned by an Executor.

    ---
    ### Your Task Inputs:
    - `instructions`: What to do.
    - `inputs`: Data or outputs from dependent tasks.
    - `notes`: Optional hints, assumptions, or constraints.
    - `success_criteria`: How to judge whether the task was successfully completed.

    ---
    ### Tool Use Policy

    You have access to a `web_search_tool` that performs a real-time internet search.

    Use this tool **only when strictly necessary**—specifically:
    - If the task instructions **explicitly require** recent or external information.
    - If you have **reasoned through the task** and cannot answer it using internal knowledge or inputs.

    **Do not use the tool by default.** Most tasks should be answerable without it.

    You are allowed to invoke the tool **no more than once per task**. If the result is insufficient, proceed with your best answer using the available information.

    Think before you search. Your efficiency and accuracy depend on it.
    
    ---
    ### What You Must Do:
    1. Read and understand the instructions.
    2. Use the inputs as needed to complete the task.
    3. Follow the notes and apply relevant constraints.
    4. Reason carefully about whether external information is needed. Use the `web_search_tool` only if it’s essential to fulfill the task accurately.
    5. Ensure your output clearly meets the success criteria.

    ---
    ### Your Output:
    Return a single response that fulfills the task instructions.
    - Be direct and informative.
    - Do not return success/failure flags—just the result.
    - Do not speculate or re-interpret the task.

    You are a focused, efficient task executor. Think before you search. Execute with precision.
  model: openai/gpt-4o-mini
  has_memory: False
  temperature: 0.3
  max_tokens: 5000
  tools:
    - web_search_tool
  output_type: TaskOutput



### Output schema definitions

This cell defines the **structured output classes** expected from each agent using `pydantic`. These schemas ensure type safety, enforce structure, and help downstream components validate and process responses.

#### Key Classes:
- `PlannerTask` – Represents a single task defined by the Planner, with fields for instructions, dependencies, success criteria, and optional notes.
- `TasksPlan` – The Planner's full output: a user-aligned goal and a sequence of structured tasks.
- `TaskOutput` – Captures a single Worker result, including the output and any execution errors.
- `OrchestratorResponse` – Used by the Executor's tool to return results from executing all tasks.
- `ExecutorResponse` – The final system output: includes a status flag, a synthesized or extracted response, and optional reasoning.

Each class uses field-level descriptions that align with agent instructions, ensuring schema expectations and role behavior remain in sync.

#### Output Registry
The `output_registry` dictionary maps class names to classes so the correct schema can be dynamically loaded based on the `output_type` string defined in each agent’s YAML config.

This decouples the agent configuration from the implementation and simplifies agent creation.

In [ ]:
# Define structured output types for Planner

from pydantic import BaseModel, Field
from typing import Dict, Literal, Optional

class PlannerTask(BaseModel):
    id: str = Field(..., pattern=r'^task-\d{3}$', description='Unique, sequential task ID (e.g., task-001, task-002)')
    instructions: str = Field(..., description='Specific instruction to be passed to an Executor.')
    success_criteria: str = Field(..., description='An objective condition or check that defines what it means for this task to be successfully completed.')
    inputs: list[str] = Field(default_factory=list, description='List any task dependencies (task IDs) or required outputs from other tasks.')
    notes: Optional[str] = Field(None, description='Optional field for hints, assumptions, or constraints.')

class TasksPlan(BaseModel):
    goal: str = Field(..., description='Your interpretation of the original objective.'),
    plan: list[PlannerTask] = Field(default_factory=list, description='Set of tasks that collectively fulfill the objective.')


# Define structured output types for Orchestrator

class TaskOutput(BaseModel):
    id: str = Field(..., description='Task ID of executed task.')
    output: str = Field(None, description='Output result from executing the task.')
    errors: Optional[str] = Field(None, description='Description of errors encountered (if any) while executing the task.')

class OrchestratorResponse(BaseModel):
    tasks_executed: Dict[str,TaskOutput] = Field(default_factory=dict, description='Outputs from executed tasks.')
    # tasks_executed: list[TaskOutput] = Field(default_factory=list, description='Outputs from executed tasks.')


# Define the structured output for the Executor

class ExecutorResponse(BaseModel):
    status: Literal["success", "partial", "failed"]
    final_output: str = Field(None, description="The best response to the Planner's goal synthesized from the task results.")
    reasoning: Optional[str] = Field(None, description="Explanation in case you return a 'partial' or 'failed' status.")
    # task_outputs: list[TaskOutput] = Field(..., description='Outputs from executed tasks.')



output_registry = {cls.__name__: cls for cls in [
    ExecutorResponse,
    OrchestratorResponse,
    PlannerTask,
    TaskOutput,
    TasksPlan,
]}

### Load agent configuration

This cell reads the `agents-config.yaml` file and parses it into a Python dictionary. The result is printed as formatted JSON to verify that the agent definitions loaded correctly.

In [ ]:
# Load the agents-config.yaml file

import json
import yaml

with open('./tmp/agents-config.yaml', 'r') as file:
    agent_config_data = yaml.safe_load(file)

formatted_json = json.dumps(agent_config_data, indent=4)
print(formatted_json)

### Tool definitions

This cell defines two tools used by agents in the Planner–Executor–Worker pattern and registers them in a `tool_registry` so they can be referenced dynamically from YAML.

Each tool is wrapped with the `@function_tool` decorator to expose it to agents. The `tool_registry` maps tool names (as defined in YAML) to the actual function objects used at runtime.

#### 🔄 `orchestrate_tasks`

Used by the **Executor**, this tool receives a `TasksPlan` and:
1. Builds a task graph using `inputs` as dependencies.
2. Executes tasks in parallel when possible.
3. Waits to run dependent tasks until their inputs are resolved.
4. Uses `assign_task()` to call a Worker agent with fully prepared task context.
5. Tracks and stores each output in `OrchestratorResponse.tasks_executed`.

It ensures correct execution order and returns a complete set of results for the plan.

#### 🌐 `web_search_tool`

Used by **Workers** to perform real-time internet searches via the Serper API.  
The tool sends a query and returns JSON search results (or `None` if an error occurs).  
Workers are instructed to use this tool sparingly and only when necessary.

In [ ]:
import asyncio
import requests
from agents import Runner, function_tool, trace
from collections import defaultdict
from typing import Any, Dict, Set

@function_tool
async def orchestrate_tasks(task_plan: TasksPlan) -> OrchestratorResponse:
    """
    Pass a plan of tasks to an orchastrator for execution and receive the results of all executed tasks.
    """

    print(f"Started orchestrate_tasks tool")
    if task_plan is None:
        raise ValueError("Cannot orchestrate an empty task plan.")

    completed = OrchestratorResponse()
    if len(task_plan.plan) < 1:
        return completed

    print(f"There are {len(task_plan.plan)} tasks in the plan.")
    task_map = {task.id: task for task in task_plan.plan}
    running: Set[str] = set()
    dependents = defaultdict(list)
    dependency_count = {task.id: len(task.inputs) for task in task_plan.plan}

    # Build reverse dependency map
    for task in task_plan.plan:
        for dep in task.inputs:
            dependents[dep].append(task.id)

    # Track tasks ready to run (no unresolved dependencies)
    ready = [task_id for task_id, count in dependency_count.items() if count == 0]

    async def run_task(task_id: str):
        print(f"running {task_id}")
        task = task_map[task_id]
        resolved_inputs = {dep: completed.tasks_executed[dep] for dep in task.inputs}
        prompt = (
            f"Task Instructions:\n{task.instructions}\n\n"
            f"Success Criteria:\n{task.success_criteria}\n\n"
            f"Inputs:\n{resolved_inputs if resolved_inputs else 'None'}\n\n"
            f"Notes:\n{task.notes or 'None'}"
        )
        result = await assign_task(prompt)
        completed.tasks_executed[task_id] = result
        print(f"completed {task_id}")

        # Mark dependents as potentially ready
        for dependent in dependents[task_id]:
            dependency_count[dependent] -= 1
            if dependency_count[dependent] == 0:
                ready.append(dependent)

    # Task execution loop
    while ready:
        batch = ready.copy()
        ready.clear()
        tasks = [asyncio.create_task(run_task(task_id)) for task_id in batch]
        for coro in asyncio.as_completed(tasks):
            await coro

    return completed

@function_tool
def web_search_tool(query: str, timeout: int = 10) -> Optional[Dict[str, Any]]:
    """
    Performs a web search using the Google Serper API.

    Args:
        query (str): The search query string.
        timeout (int): The number of seconds to wait for a response from the server.

    Returns:
        Optional[Dict[str, Any]]: A dictionary containing the JSON response from the API,
                                  or None if an error occurs.
    """

    # Validate API key
    api_key = os.getenv('SERPER_API_KEY')
    if not api_key:
        print("Error: SERPER_API_KEY environment variable not set.")
        return None


    with requests.Session() as session:
        url = "https://google.serper.dev/search"

        headers = {
            'X-API-KEY': api_key,
            'Content-Type': 'application/json'
        }

        print(f"web_search_tool: {query}")
        payload = {"q": query}

        try:
            response = session.post(url, headers=headers, json=payload, timeout=timeout)

            # Check for successful HTTP status code
            response.raise_for_status()
            
            return response.json()

        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
            print(f"Response content: {response.text}")
        except requests.exceptions.ConnectionError as conn_err:
            print(f"Connection error occurred: {conn_err}")
        except requests.exceptions.Timeout as timeout_err:
            print(f"Timeout error occurred: {timeout_err}")
        except requests.exceptions.RequestException as req_err:
            print(f"An unexpected error occurred: {req_err}")
        except json.JSONDecodeError as json_err:
            print(f"Error decoding JSON response: {json_err}")
            print(f"Response content: {response.text}")

    return None



tool_registry = {tool.name: tool for tool in [
    orchestrate_tasks,
    web_search_tool
]}

### Instantiate agents from configuration

This cell defines `create_agent()`, a utility that instantiates agents based on their definitions in `agents-config.yaml`.

🔧 **How it works:**
- Validates that the requested agent type is defined in the YAML file (e.g., planner, executor, worker).
- Loads the model, temperature, max_tokens, and instructions from YAML.
- Resolves `tools` and `output_type` using internal registries.
- Prepends today's date to the agent's instructions (for context-aware prompting).
- Optionally enables session memory via `SQLiteSession` (not used in this pattern).

At the end of the cell, the Planner, Executor, and Worker agents are instantiated and ready for use.

In [ ]:
from datetime import datetime
from agents import Agent, ModelSettings, SQLiteSession

def create_agent(agent_type: str = None):
    """ 
    Creates and returns an Agent that matches the given definition in the agents-config 
    YAML file. Optionally returns a memory Session if agent configuration calls for it.
    """

    if agent_type is None or not agent_type.strip():
        raise ValueError("agent_type must be a valid type of agent defined in agent-configs.yaml.")
    
    agent_config = agent_config_data.get(agent_type)
    if agent_config is None:
        raise ValueError(f"'{agent_type}' does not match an agent defined in agent-configs.yaml.")

    # Generate a timestamp string for unique naming
    now_string = datetime.now().strftime("%Y-%m-%dT%H:%M:%SU%s")

    # Prepare model settings
    agent_model_settings = ModelSettings(
        temperature=agent_config.get('temperature'),
        max_tokens=agent_config.get('max_tokens'),
    )

    # Resolve tools from string names (if any)
    tool_names = agent_config.get('tools', [])
    resolved_tools = []
    for name in tool_names:
        tool = tool_registry.get(name)
        if tool is None:
            raise ValueError(f"Tool '{name}' specified in config for agent '{agent_type}' is not registered.")
        resolved_tools.append(tool)

    # Instantiate the agent
    agent_name = agent_config.get('name') or f"{agent_type}_{now_string}"
    agent_instructions = agent_config.get('instructions')
    if agent_instructions is None or not agent_instructions.strip():
        raise ValueError(f"No instructions for '{agent_name}' have been specified in the configuration.")
    new_agent = Agent(
        name=agent_name,
        instructions=f"Today is {datetime.now().strftime('%Y-%m-%d')}.\n\n{agent_instructions}",
        tools=resolved_tools,
        model=agent_config.get('model') or agent_model_DEFAULT,
        output_type=output_registry.get(agent_config.get('output_type') or None),
        model_settings=agent_model_settings
    )

    # Create memory session for agent if configured
    agent_has_memory = agent_config.get('has_memory') or False
    agent_session_name = f"{agent_name}__SESSION_{now_string}" if agent_has_memory else None
    agent_session = SQLiteSession(agent_session_name) if agent_session_name else None

    return (new_agent, agent_session)

# Instantiate the agents. We're not using memory so ignore the second return.
planner, _ = create_agent('planner')
executor, _ = create_agent('executor')
worker, _ = create_agent('worker')


### Assign a task to the Worker agent

This cell defines the `assign_task()` function, which asynchronously routes a task prompt to the Worker agent and returns its structured output as a `TaskOutput` object. This function is used internally by the `orchestrate_tasks` tool to execute individual atomic tasks.

- It uses `Runner.run()` to execute the prompt against the Worker.
- The `enable_trace` flag controls whether the task execution is wrapped in a trace context. This is typically set to `False` because the Worker is usually called from within the Executor’s trace.
- Tracing can be enabled manually for standalone debugging.

In [ ]:
# Assign a task to a worker agent

from agents import Runner, function_tool, trace

# @function_tool
async def assign_task(task: str, enable_trace: bool = False) -> TaskOutput:
    """
    Asssign a task to the agent and receive its response in return.
    """

    # Check that Worker agent exists
    if worker is None:
        raise ValueError("Agent has not been created yet.")
    
    # When calling this function standalone, set enable_trace to True
    if enable_trace:
        with trace(worker.name):
            result = await Runner.run(worker, task)
    else:
        result = await Runner.run(worker, task)
    
    return result.final_output_as(TaskOutput)


### Optional: Test the Worker with a sample task

This cell defines a toggleable test for the `assign_task()` function. It is disabled by default to avoid unnecessary API usage and latency. Enable it to test Worker behavior.

When `enable_worker_task_test` is set to `True`, it sends a sample task to the Worker agent and prints the result.

🧪 **Purpose:**  
- Verify the Worker is correctly instantiated.
- Confirm that tool usage (like `web_search_tool`) is functioning as expected.
- Provides a quick sanity check without running the full Planner–Executor pipeline.

In [ ]:
enable_worker_task_test = False     # Set to True to run this cell

if enable_worker_task_test:
    test_task = {
        'id': 'task-017',
        'instructions': 'Tell me today\'s date and list the top 10 headlines around the world. DO NOT INVENT THEM.',
        'success_criteria': 'Response contains a valid date and lists 10 news headlines.',
        'inputs': None
    }

    task_string = json.dumps(test_task)
    print(task_string)

    result = await assign_task(task_string, True)

    if result is not None:
       print(f"TaskOutput:\n{result.model_dump_json(indent=2)}")

### Generate a task plan from a user query

The next two cells define and optionally run the Planner agent to break down a high-level user query into a structured task plan. The returned plan is used by the Executor to drive downstream task execution.

🔧 **`make_tasks_plan(query)`**
- Sends the query to the Planner using `Runner.run()`.
- Wraps the execution in a trace for observability.
- Returns the result as a `TasksPlan` object.

🧪 **Optional test**
- The second cell allows you to test the Planner by providing a real query (disabled by default).
- When enabled, it prints the Planner’s full task plan in JSON format.

In [ ]:
# Run a user query

from agents import Runner, function_tool, trace

async def make_tasks_plan(query: str) -> TasksPlan:

    # Check that Planner agent exists
    if planner is None:
        raise ValueError("Agent has not been created yet.")
        
    result = None

    with trace(planner.name):
        result = await Runner.run(planner, f"User Goal: {query}")

    return result.final_output_as(TasksPlan)
    

In [ ]:
enable_planner_query_test = False     # Set to True to run this cell

if enable_planner_query_test:
    q = """Produce a briefing document summarizing the most significant developments in climate policy across the US, EU, 
        and China over the past 12 months. Include key policy changes, notable legislation, and international agreements. 
        Conclude with a comparative analysis highlighting similarities and differences."""
    plan = await make_tasks_plan(q)

    if plan is not None:
        print(f"TasksPlan:\n{plan.model_dump_json(indent=2)}")

### Execute a task plan using the Executor agent

These cells handle the execution of a full task plan by coordinating with the Executor agent. Running the test requires `plan` to be a valid `TasksPlan` -- just run the planner query test in the previous cell first.

🧩 **`execute_tasks_plan(plan)`**
- Accepts a `TasksPlan` (from the Planner).
- Serializes the plan to JSON and sends it to the Executor via `Runner.run()`.
- The Executor uses the `orchestrate_tasks` tool to run all tasks, handle dependencies, and collect final outputs.
- Returns a structured `ExecutorResponse` with:
  - `status`: Whether the goal was fulfilled (`success`, `partial`, `failed`)
  - `final_output`: Synthesized response to the original user query
  - `reasoning`: Optional, for partial/failed responses

🧪 **Optional test**
- The second cell lets you test this execution process using a previously defined `plan` (disabled by default to prevent accidental API calls).
- If enabled, it prints the final status and output from the Executor.


In [ ]:
from agents import Runner, trace

async def execute_tasks_plan(plan: TasksPlan) -> ExecutorResponse:

    # Check that Planner agent exists
    if executor is None:
        raise ValueError("Agent has not been created yet.")
        
    plan_str = plan.model_dump_json()

    result = None

    with trace(executor.name):
        result = await Runner.run(executor, plan_str)

    if result and result.final_output:
        return result.final_output_as(ExecutorResponse)
    else:
        raise ValueError("The runner did not return a valid final_output")
        

In [ ]:
enable_execute_tasks_plan_test = False     # Set to True to run this cell

if enable_execute_tasks_plan_test:
    if not plan:
        raise ValueError("`plan` must be a valid TasksPlan to run this test")

    answer = await execute_tasks_plan(plan=plan)

    if answer is not None:
        print(f"Result Status: {answer.status}")
        print(f"Final Output:\n{answer.final_output}")

In [ ]:
from agents import Runner, trace

async def run(query: str) -> ExecutorResponse:
    
    # Check that Planner agent exists
    if planner is None:
        raise ValueError("Agent has not been created yet.")
        
    # Make a plan from the user query
    with trace(planner.name):
        planner_result = await Runner.run(planner, f"User Goal: {query}")
    if planner_result is None or not planner_result.final_output:
        raise ValueError("Planner agent failed to produce a valid plan.")

    # Serialize the plan         
    plan = planner_result.final_output_as(TasksPlan)
    plan_str = plan.model_dump_json()

    # Execute the plan
    with trace(executor.name):
        executor_result = await Runner.run(executor, plan_str)

    if executor_result and executor_result.final_output:
        return executor_result.final_output_as(ExecutorResponse)
    else:
        raise ValueError("The Executor did not return a valid response.")
    

In [ ]:
test_query = [
    """Produce a briefing document summarizing the most significant developments in climate policy across the US, EU, 
    and China over the past 12 months. Include key policy changes, notable legislation, and international agreements. 
    Conclude with a comparative analysis highlighting similarities and differences.
    - Unless instructed otherwise, synthesize the final output as a concise narrative document. 
    - In the final output, prefer paragraphs over lists; only use bullets and numbered lists when absolutely necessary.
    - Format the final output as a markdown document.
    """,

    """
    Produce a policy profile on India's renewable energy transition over the past five years. The output should include:
    - A timeline of major policy actions and reforms.
    - Summary of government programs (e.g., subsidies, R&D funding).
    - Key statistics on solar, wind, and hydro adoption.
    - Commentary from recent policy papers and media sources.

    Synthesize all findings into a single markdown document with clearly delineated sections. Use a table for the 
    timeline. Do not include citations—just summarize the most critical content.
    """,

    """
    Summarize the current state of the global semiconductor supply chain. Include:
    - Major manufacturers and suppliers by region.
    - Current geopolitical and economic risks.
    - Recent legislation or trade agreements (past 2 years).
    - Comparative analysis of US, Taiwan, South Korea, and China.

    Present the final result as a well-structured briefing with a neutral tone. Ensure that all key claims are grounded in recent facts and avoid vague generalizations. Use paragraphs only—do not use bullet points or headings.
    """,

    """
    Extract and transform insights from the following workflow:
    1. Identify five notable research papers on large language models (LLMs) from the last 18 months.
    2. Summarize the key contributions and limitations of each paper.
    3. Compare the approaches used and identify recurring challenges or patterns.

    Produce a structured markdown report with three sections: Overview, Comparative Insights, and Conclusions. Use clear transitions between sections. Avoid technical jargon when possible.
    """,

    """ 
    Write a historical summary of the Paris Agreement: how it was formed, what it aims to achieve, and how its implementation has evolved over time. Focus on pivotal moments from 2015 to present.

    Output should be a standalone markdown article. No need to break into multiple tasks unless truly necessary.
    """,

    """ 
    Evaluate how five major tech companies (e.g., Apple, Google, Microsoft, Meta, Amazon) are addressing AI ethics and responsible AI governance. For each company:
    - Identify key public commitments, policies, or frameworks.
    - Assess how comprehensive and enforceable their commitments appear.

    Synthesize the findings into a comparative markdown report and assign a qualitative score (e.g., High, Medium, Low) to each company’s efforts. Present the final output in a table followed by a short narrative summary.
    """,

    """
    blah blah blah
    """
]

q = test_query[6]

answer = await run(query=q)

if answer is not None:
    print(f"Query: {q}")
    print(f"Result Status: {answer.status}")
    print(f"Final Output:\n{answer.final_output}")